### Network MultiAgent

In Multiagent Architecture, Agents are represented as Graph Nodes. A common pattern in MultiAgent interaction is HANDOFF's, where one agent handoff control to another

In  Network MultiAgents: Agents will collabrate with each other for executing the flow.
Agent Will handoff the response to another agent.

We use `Command` for sending the handoff to another agent and updating the messages state.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
llm.invoke("hi hello how are you?")

AIMessage(content="Hello! I'm here to help, so feel free to ask any questions or let me know how I can assist you today. How are you doing?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-Bs4IqXOH9mC41T92QL7HSi7LT4uE6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5f9ebfcb-ca43-4699-86b3-ea29e95a0696-0', usage_metadata={'input_tokens': 13, 'output_tokens': 30, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent

def add_number(state):
    result=state["num1"]+state["num2"]
    print(f"addition is {result}")
    return Command(goto="multiply",update={"sum":result})

state={"num1":10,"num2":20}
add_number(state)

addition is 30


Command(update={'sum': 30}, goto='multiply')

In [6]:
from langchain_core.tools import tool

@tool
def transfer_to_multiplication_expert():
    """Ask multiplication agent for help"""
    print("Transferring to multiplication expert...")
    return

In [7]:
@tool
def transfer_to_addition_expert():
    """Ask addition agent for help"""
    print("Transferring to addition expert...")
    return

In [8]:
from typing_extensions import Literal
from langgraph.graph import MessagesState,StateGraph, START,END

## Agent1
def additional_expert(state:MessagesState)-> Command[Literal["multiplication_expert", "__end__"]]:
    
    system_prompt = (
        "You are an addition expert, you can ask the multiplication expert for help with multiplication."
        "Always do your portion of calculation before the handoff."
    )
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    ai_msg = llm.bind_tools([transfer_to_multiplication_expert]).invoke(messages)
    
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        
        return Command(
            goto="multiplication_expert", update={"messages": [ai_msg, tool_msg]}
        )
    return {"messages": [ai_msg]}

In [9]:
# Agent2
def multiplication_expert(state:MessagesState)-> Command[Literal["additional_expert", "__end__"]]:
    
    system_prompt = (
        "You are a multiplication expert, you can ask an addition expert for help with addition. "
        "Always do your portion of calculation before the handoff."
    )
    
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    
    ai_msg = llm.bind_tools([transfer_to_addition_expert]).invoke(messages)
    
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        return Command(goto="additional_expert", update={"messages": [ai_msg, tool_msg]})
    return {"messages": [ai_msg]}

In [12]:
graph=StateGraph(MessagesState)
graph.add_node("additional_expert",additional_expert)
graph.add_node("multiplication_expert",multiplication_expert)
graph.add_edge(START, "additional_expert")
app=graph.compile()

app.invoke({"messages":[("user","what's (3 + 5) * 12. Provide me the output")]})

{'messages': [HumanMessage(content="what's (3 + 5) * 12. Provide me the output", additional_kwargs={}, response_metadata={}, id='42f428c4-d0b7-42ca-b9d6-b721dbb6156b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tiqih2MTHRw0kUJ545hUJoai', 'function': {'arguments': '{}', 'name': 'transfer_to_multiplication_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 86, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'id': 'chatcmpl-Bs4OJWOn9sQSc8fGzyBBpg3vfGhFz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7e9024be-b93b-41b1-af8c-4099f531fca3-0', tool_calls=[{'name': 'transfer_to_multiplication_expert', 'args': {},